In [1]:
#------------------------------------------------------------------------------------------------------------------------------
# import dependencies
#------------------------------------------------------------------------------------------------------------------------------
import re
import json
import sys, os

import requests
import warnings
import psycopg2

from pprint import pprint
from copy import deepcopy

from country_list import countries_for_language

In [2]:
#------------------------------------------------------------------------------------------------------------------------------
# generate artist list
#------------------------------------------------------------------------------------------------------------------------------
"""get artists list csv"""
def get_artists(countries):  
    dicy = {}
    limit = 1000    # SPECIFY
    urlraw = r'https://ws.audioscrobbler.com/2.0/?method=geo.gettopartists&country={}&api_key=70894b101e887cf7fd9443bd5067f675&format=json&limit={}'
    
    for cty in countries.values():
        r = requests.get(urlraw.format(cty, limit))
        tmp = r.json()
#         pprint(tmp)

        try:
            for i in range(len(tmp['topartists']['artist'])):
                artistname = tmp['topartists']['artist'][i]['name']

                dicy[artistname] = 1
        except:
            print(f'top artists for {cty} CANNOT be found')
            pass
        
    return dicy


def create_glbartist():
    countriesDict = dict(countries_for_language('en'))
#     print(countriesDict)
    dicy = get_artists(countriesDict)
    # print(len(dicy), dicy)

    with open('glbartists.csv', 'w', encoding='utf-8') as fw:
        print('artists', file=fw)

        for artistname in dicy:
            print(artistname, file=fw)
            print(f'created... {artistname}')

In [3]:
# create_glbartist()

In [4]:
#------------------------------------------------------------------------------------------------------------------------------
# populate artist details
#------------------------------------------------------------------------------------------------------------------------------
"""get artist info"""
def get_artist_info(artist):
    dicy = {}
    pot = {}

    urlraw = r'https://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={}&api_key=70894b101e887cf7fd9443bd5067f675&format=json'

    try:
        r = requests.get(urlraw.format(artist))
        tmp = r.json()
#         pprint(tmp)

        artistgenre = tmp['artist']['tags']['tag']
#         print(artist, artistgenre)
    except:
        print(f"""your artist ({artist}) is WRONG""")
        pass
    else:
        for i in range(len(artistgenre)):
            if artistgenre[i]['name'] != artist:
                addartist = artistgenre[i]['name'].replace('-', ' ').title()

                pot[addartist] = 1

        genre = ', '.join(pot)

        playcount = tmp['artist']['stats']['playcount']

        listeners = tmp['artist']['stats']['listeners']

        try:
            playcountperlistener = str(int(int(playcount)/int(listeners)))
        except:
            playcountperlistener = ''

        artistinfo = re.sub(rf'(?= <a).*?(?=$)', '', tmp['artist']['bio']['summary']).replace('\n', ' ').strip()
        artistinfo = artistinfo.replace('|', ',').replace('"', '').replace("'", "")
        
        artistlink = tmp['artist']['bio']['links']['link']['href']

    #     print(genre)
    #     print(playcount)
    #     print(listeners)
    #     print(artistinfo)
    #     print(artistlink)

        return f'{genre}|{playcount}|{listeners}|{playcountperlistener}|{artistinfo}|{artistlink}'


def create_glb_artistbio():
    fw = open('glbartistsbio.csv', 'w', encoding='utf-8')

    with open('glbartists.csv', 'r', encoding='utf-8') as fr:
        print('artistname|genre|playcount|listeners|playcountperlistener|artistinfo|artistlink', file=fw)

        next(fr)

        for artistname in fr:
            artistname = artistname.strip()
    #         print(artistname)

            info = get_artist_info(artistname)
        
            artistname = artistname.replace('|', ',').replace('"', '').replace("'", "")
        
            if info != None:
                artistbio = f'{artistname}|{info}'
                print(artistbio, file=fw)

                print(f'completed... {artistname}')

    fw.close()

In [5]:
# create_glb_artistbio()

In [6]:
# get_artist_info('#1 Dads')

In [7]:
#------------------------------------------------------------------------------------------------------------------------------
# high lv fns
#------------------------------------------------------------------------------------------------------------------------------
def search_sim_artist(artist):
    params = {
        'host': 'localhost',
        'database': 'artistsbio_db',
        'user': 'postgres',
        'password': 'Cw18745.,'
    }
    conn = psycopg2.connect(**params)

    with conn:
#         sql = f"select artistname from public.glbartistbio where artistname ~* '{artist}' limit 5;"
        sql = f"select artistname from glbartistbio where artistname ~* '{artist}' and artistname !~* 'feat' \
            order by playcount::int desc limit 5;"
        
        cur = conn.cursor()
        cur.execute(sql)

    print('ARE YOU SEARCHING FOR?')
    for r in cur:
        artistname = r[0]
        print(r[0])

    cur.close()

    conn.commit()
    conn.close()
    
    
def search_artist_bio(artist):
    params = {
    'host': 'localhost',
    'database': 'artistsbio_db',
    'user': 'postgres',
    'password': 'Cw18745.,'
    }
    conn = psycopg2.connect(**params)

    with conn:
#         sql = f"select artistname from public.glbartistbio where artistname ~* '{artist}' limit 5;"
        sql = f"select * from glbartistbio where artistname ~* '{artist}' and artistname !~* 'feat' \
            order by playcount::int desc limit 5;"
        
        cur = conn.cursor()
        cur.execute(sql)

        rec = [r for r in cur]
        
        cur.close()
        
        for i in range(len(rec)):
            print('-' * 100)
            print(f"""\
INTRO TO YOUR ARTIST - {rec[i][1]}:
Genres: {rec[i][2]}
Playcounts: {int(rec[i][3]):,}
Listeners: {int(rec[i][4]):,}
Playcounts per listener: {rec[i][5]}
Bio: {rec[i][6]}
{rec[i][7]}""")
            
    print('-' * 100)
    
    conn.commit()
    conn.close()

In [8]:
search_sim_artist('justin')

ARE YOU SEARCHING FOR?
Justin Bieber


In [9]:
search_artist_bio('justin')

----------------------------------------------------------------------------------------------------
INTRO TO YOUR ARTIST - Justin Bieber:
Genres: Pop, Black Metal, Justin Bieber, Rnb, Hip Hop
Playcounts: 103,652,355
Listeners: 2,418,961
Playcounts per listener: 42
Bio: Justin Drew Bieber (born March 1, 1994 in London, Ontario, Canada) is a Canadian pop musician, actor, and singer-songwriter. Bieber was discovered in 2008 by American talent manager Scooter Braun, who came across Biebers videos on YouTube and later became his manager. Braun arranged for him to meet with entertainer Usher Raymond in Atlanta, Georgia. Bieber was signed to Raymond Braun Media Group (RBMG), and then to an Island Records recording contract offered by record executive L.A. Reid.
https://last.fm/music/Justin+Bieber/+wiki
----------------------------------------------------------------------------------------------------
